In [26]:
import sys
sys.path.append('../sample/')
from ann import Perceptron, NeuralNetwork

sys.path.append('../../')
import little_mcmc.sample.mcmc as mc

import numpy as np
from random import gauss, random
from time import time
import matplotlib.pyplot as plt

Typing Hint:

In [27]:
from typing import List, Tuple, Mapping

Array = np.array(List[float])
Value = float

### Set global variables:

In [36]:
# Ann
net_size = [2, 1]
input_size = 1


#MCMC
step_length = 0.05
mcmc_threshold = 0.01

chain_num = 1
goal = 0.001
iterations = 10 ** 3

### Set error-function as, e.g.,

In [37]:
def error_function(outputs: Array, targets: Array) -> float:
    
    assert len(outputs) == len(targets)
    
    return 0.5 * np.sum(np.square(outputs - targets))

## Aim: fit `[0.5 * sin(x) for x in np.linspace(-7, 7, 20)]`.

### Preparing for MCMC

In [38]:
def random_move(net: NeuralNetwork, step_length=step_length) -> NeuralNetwork:
    result_net = net.copy()
    
    for layer in result_net.layers:
        for perceptron in layer:
            perceptron.weights = np.array([weight + gauss(0, 1) * 1
                                           for weight in perceptron.weights
                                          ])
    return result_net

In [34]:
p = Perceptron(5)
print(p.weights)

p.weights = [1 for weight in p.weights]
print(p.weights)

[ 0.01193646 -0.03305072 -0.01886215  0.02916268  0.02731315  0.01411343]
[1, 1, 1, 1, 1, 1]


In [43]:
result_net = init_net.copy()
    
result_net.layers[0][0].weights = np.array([1 for w in p.weights])
print(result_net.layers[0][0].weights)        
result_net.weights

[1 1]


[[array([ 0.02507894,  0.0201661 ]), array([-0.03505475,  0.03984515])],
 [array([-0.03317513, -0.02975945,  0.02151422])]]

In [39]:
init_net = NeuralNetwork(net_size, input_size)
print(init_net.weights)

print()
new_net = random_move(init_net, 1)
print(new_net.weights)

[[array([ 0.02507894,  0.0201661 ]), array([-0.03505475,  0.03984515])], [array([-0.03317513, -0.02975945,  0.02151422])]]

[[array([ 0.02507894,  0.0201661 ]), array([-0.03505475,  0.03984515])], [array([-0.03317513, -0.02975945,  0.02151422])]]


In [7]:
init_net = NeuralNetwork(net_size, input_size)
result_net = init_net.copy()

layer = result_net.layers[0]
perceptron = layer.perceptrons[0]
print(perceptron.weights)
perceptron.weights = [__ + gauss(0, 1) * 1 for __ in perceptron.weights]
print(perceptron.weights)

AttributeError: 'list' object has no attribute 'perceptrons'

Function to maximize:

In [ ]:
# Input
x = np.linspace(-7, 7, 20)

# Target
y = np.abs(np.sin(x)) * 0.5


def f(net: NeuralNetwork, inputs=x, targets=y) -> Value:
        
    outputs = np.array([net.output([__])[0] for __ in inputs])
    
    return error_function(outputs, targets)

### New MCMC

In [ ]:
def grad(f, net, delta=0.001, threshold=0):
    
    def move_along_weight_axis(net, position):
        
        result_net = net.copy()
        
        layer = position[0]
        perceptron = position[1]
        weight = position[2]
        
        result_net.layers[layer].perceptrons[perceptron].weights[weight] += delta
        
        return result_net
    
    result = []
    
    f_value_0 = f(net)
    
    for i, layer in enumerate(net.layers):
        
        result_p = []
        
        for j, perceptron in enumerate(layer.perceptrons):
            
            result_w = []
            
            for k, weight in enumerate(perceptron.weights):
                
                position = [i, j, k]
                f_value_1 = f(move_along_weight_axis(net, position))
                
                delta_f = (f_value_1 - f_value_0) / delta
                
                if abs(delta_f) < threshold:
                    result_w.append(0)
                    
                else:
                    result_w.append(delta_f)
                
            result_p.append(result_w)
            
        result.append(result_p)
        
    return result

def minQ(f, net, dela=0.001, tolerence=0.001):
    g = grad(f, net, delta)
    
    result = True
    
    for i in range(len(g)):
        for j in range(len(g[i])):
            for k in range(len(g[k])):
                
                if abs(g[i][j][k]) < tolerence:
                    return False
    
    return result

In [ ]:
def mcmc(f, random_move, step_length, init_net0, goal, iterations, threshold=mcmc_threshold):
    
    
    weights = init_net0.weights.copy()
    #num_weights = len(np.array(weights).reshape(1,))
    
    result = []
    
    init_net = init_net0.copy()
    init_value = f(init_net)
        
    for step in range(iterations):
        
        reach_the_goal = (abs(init_value) < goal)
        
        if reach_the_goal:

            print('Reach the goal: ', goal)
            break
        
        else:
            
            new_net = random_move(init_net, step_length)
            new_value = f(new_net)
            
            moveQ = ((new_value - init_value) / (np.sqrt(num_weights) * step_length) < -random() * threshold * 0)
            
            if moveQ:
                init_net = new_net.copy()
                init_value = new_value
                
                result.append((new_net, new_value))
                
            else:
                pass
            
        if step == iterations - 1:
            print('Had not reach the goal when iteration = {0} was exhausted.'.format(iterations))
                
    return result

## Test Single Chain MCMC

In [ ]:
init_net = NeuralNetwork(net_size, input_size)
print(grad(f, init_net))

aaa = mcmc(f, random_move, 0.1, init_net, 0.001, 10 ** 3)

In [ ]:
len(aaa)

In [ ]:
new_net = random_move(init_net, 0.05)

print(init_net.weights)
print(new_net.weights)
print()
print(grad(f, init_net, threshold=0.001))
print(grad(f, new_net, threshold=0.001))

In [ ]:
print(aaa[-1][0].weights)
print()
print(grad(f, aaa[-1][0]))

### Do MCMC

In [ ]:
# Do mcmc
chain_list = []

t_begin = time()

for step in range(chain_num):
    init_net = NeuralNetwork(net_size, input_size)

    net_chain = mcmc(f, random_move,
                     init_net, 0.001,
                     10 ** 5
                     )
    chain_list.append(net_chain)

bc = mc.best_chain(chain_list)
best_net = bc[-1][0]
ef_value = bc[-1][1]

t_end = time()
print('time spent by fitting: ', t_end - t_begin)

In [ ]:
len(chain_list[0])

### Show Result

In [ ]:
error_function_values = [__[1] for __ in bc]
plt.plot([i for i in range(len(error_function_values))],
         error_function_values)
plt.xlabel('chain_node of best_chain')
plt.ylabel('-1 * error_function_value')
plt.show()


plot_x = np.linspace(-6.0,6.0,150)
plot_y_ann = [best_net.output([__]) for __ in plot_x]
plot_y_target = np.abs(np.sin(plot_x)) * 0.5

plt.plot(plot_x, plot_y_target, '-',
         plot_x, plot_y_ann, '.', alpha = 0.3
        )
plt.legend(['train target', 'net output'])
plt.show()

**Far from good.**

Further exame

In [ ]:
test_net = random_move(best_net, 0.01)

print('test: ', f(test_net))
print('best: ', f(best_net))

print('test of relative gain: ', mc.relative_gain(f(best_net), f(test_net)))

### Print out Fitted Weights

In [ ]:
for i in range(len(best_net.layers[0])):
    print(best_net.layers[0][i].weights)

print()

for i in range(len(best_net.layers[1])):
    print(best_net.layers[1][i].weights)